In [11]:
# -*- coding: utf-8 -*-
"""
Created on Tue 22 August 2023

@author: Xuefei Han
"""
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

if __name__ == "__main__":
    index_df = pd.read_csv("D:/PycharmProjects/written_test/data/index_df.csv")
    index_df = index_df.rename(columns={'Unnamed: 0': "Date", 'index': "idx_price"})

    # final result dataframe yearly
    annual_stats = pd.DataFrame(
        columns=['Year', 'Annual_Return', 'Annual_Volatility', 'Sharpe_Ratio', 'Max_Drawdown', 'Max_Drawdown_Start', 'Max_Drawdown_End'])

    # convet the str date to datetime for yearly stat
    index_df['Date'] = pd.to_datetime(index_df['Date'], format='%Y%m%d')
    index_df['Year'] = index_df['Date'].dt.year

    # calculate stat for each year
    for year in index_df['Year'].unique():

        sub_df = index_df[index_df['Year'] == year]

        sub_df.loc[:, 'Daily_Return'] = sub_df['idx_price'].pct_change().fillna(0)
        if len(sub_df) < 200:
            annual_return = np.prod(1 + sub_df['Daily_Return']) - 1
        else:
            annual_return = np.prod(1 + sub_df['Daily_Return']) ** (252 / len(sub_df)) - 1
        annual_volatility = np.std(sub_df['Daily_Return']) * np.sqrt(252)
        sharpe_ratio = annual_return / annual_volatility

        # max drawdown
        sub_df.loc[:, 'Cumulative_Return'] = (1 + sub_df['Daily_Return']).cumprod()
        cumulative_max = np.maximum.accumulate(sub_df['Cumulative_Return'])
        drawdown = (cumulative_max - sub_df['Cumulative_Return']) / cumulative_max
        max_drawdown = drawdown.max()
        end_date_index = drawdown.idxmax()
        end_date = sub_df.loc[end_date_index, 'Date']
        start_date_index = sub_df.loc[:end_date_index, 'Cumulative_Return'].idxmax()
        start_date = sub_df.loc[start_date_index, 'Date']

        annual_stats.loc[len(annual_stats)] = [year, annual_return, annual_volatility, sharpe_ratio, max_drawdown, start_date, end_date]
    annual_stats.columns = ['年份', '年化收益率(%)', '年化波动率(%)', '夏普比率', '最大回撤(%)', '最大回撤开始日期', '最大回撤结束日期']
    annual_stats['最大回撤开始日期'] = annual_stats['最大回撤开始日期'].dt.strftime('%Y%m%d')
    annual_stats['最大回撤结束日期'] = annual_stats['最大回撤结束日期'].dt.strftime('%Y%m%d')
    avg=annual_stats.mean(axis=0,numeric_only=True)
    avg['年份']='all'
    annual_stats=annual_stats.append(avg,ignore_index=True)


    # print(annual_stats)


In [12]:
annual_stats

,年份,年化收益率(%),年化波动率(%),夏普比率,最大回撤(%),最大回撤开始日期,最大回撤结束日期
0,2013,0.000000,0.000000,NaN,0.000000,20131231,20131231
1,2014,0.169445,0.030948,5.475103,0.012479,20140630,20140718
2,2015,0.077397,0.028395,2.725774,0.014100,20150610,20150824
3,2016,0.018170,0.031977,0.568213,0.042556,20161128,20161220
4,2017,0.077468,0.026657,2.906080,0.020908,20170324,20170510
5,2018,0.042508,0.028375,1.498083,0.019563,20180828,20181126
6,2019,0.075555,0.024706,3.058099,0.013704,20190909,20191030
7,2020,0.074802,0.029182,2.563344,0.022729,20200306,20200319
8,2021,0.064075,0.028159,2.275439,0.012795,20210601,20210617
9,2022,-0.001391,0.029890,-0.046538,0.027560,20220124,20220315
